# Loading Dataset into Panda Dataframe

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
from azureml.core import Datastore, Dataset
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

# retrieve an existing datastore in the workspace by name
datastore_name = 'mldemoblob'
datastore = Datastore.get(ws, datastore_name)

# create a TabularDataset from the file path in datastore
datastore_path = [(datastore, 'melb_data.csv')]
tabdf = Dataset.Tabular.from_delimited_files(path=datastore_path)

# increase display of all columns of rows for panda datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# create panda dataframe
raw_df = tabdf.to_pandas_dataframe()
raw_df.head()

## Data Definition
- Rooms: Number of rooms
- Price: Price in dollars
- Method:<br> S - property sold; <br>
        SP - property sold prior;<br> 
        PI - property passed in; <br>
        PN - sold prior not disclosed;<br> 
        SN - sold not disclosed; <br>
        NB - no bid; <br>
        VB - vendor bid; <br>
        W - withdrawn prior to auction;<br> 
        SA - sold after auction; <br>
        SS - sold after auction price not disclosed.<br> 
        N/A - price or highest bid not available.

- Type:<br>   br - bedroom(s);<br> 
        h - house,cottage,villa, semi,terrace; <br>
        u - unit, duplex; <br>
        t - townhouse; <br>
        dev site - development site; <br>
        o res - other residential.
        
- SellerG: Real Estate Agent
- Date: Date sold
- Distance: Distance from CBD
- Regionname: General Region (West, North West, North, North east …etc)
- Propertycount: Number of properties that exist in the suburb.
- Bedroom2 : Scraped # of Bedrooms (from different source)
- Bathroom: Number of Bathrooms
- Car: Number of carspots
- Landsize: Land Size
- BuildingArea: Building Size
- CouncilArea: Governing council for the area

# Data Exploration

In [ ]:
# The shape shows us the number of columns (features/pot. labels)= 21 and the number of rows (samples) = 13580
raw_df.shape

In [ ]:
# calculate Unique Values, Missing Values Percentage, Percentage of Values in the biggest category and Datatype
stats = []
for cl in raw_df.columns:
    stats.append((cl, 
                  raw_df[cl].nunique(), 
                  raw_df[cl].isnull().sum(),
                  raw_df[cl].isnull().sum() * 100 / raw_df.shape[0],
                  raw_df[cl].value_counts(normalize=True, dropna=False).values[0] * 100,
                  raw_df[cl].dtype))

# create new dataframe containing the above mentioned stats    
stats_df = pd.DataFrame(stats, columns=['Feature', 
                                        'Unique Values',
                                        'Missing Values',
                                        'Missing Values [%]',
                                        'Values in the biggest category [%]',
                                        'Datatype'])

stats_df.sort_values('Missing Values [%]', ascending=False)

First look at the above results. What do we see?

- we seem to have 4 features with missing values (BuildingArea, YearBuilt, CouncilArea, Car)

- looking at the datatypes, there seem to be a lot of float64, even though a lot of them are very small integer numbers, like YearBuilt, Car, Bathroom, Bedroom2, Postcode, Price. Float64 stores as the name suggest data in 64-bit. On top of that, but probably not a problem in this case, it can only represent a real number to a certain extent of precision. Either way, most of these are small natural numbers, which we could store in int32 to save space.

- there are 7 features of type Object, which means they are probably strings. We should have a look at them.<br>
        Type has 3 distinct values, our definitions shows 6<br>
        Method has 5 distinct values, our definition shows 11<br>
        SellerG has 268 distinct seller names<br>
        Address has 13378 distinct values, but we have 13580 samples, so there seems to be places with the same address<br>
        Regionname has 8 distinct values, the regions of Melbourne<br>
        Subburb has 314 distinct values, the suburbs of Melbourne<br>
        CouncilArea has 33 distinct values, and is the only categorical feature with missing values<br>

- we see there is a column called Price, which might be a good label/target for supervised training.

Before doing anything else, lets clean some names and get rid of some features that might be not of too much interest for our first analysis.

# First Data Cleansing

In [ ]:
# Let's remove the Address and the Seller for now (we can add them later back into the mix)
df = raw_df.drop(['Address', 'SellerG'],axis=1)
# rename some of the columns
df = df.rename(columns={'Bedroom2': 'Bedrooms', 'Bathroom': 'Bathrooms','Regionname': 'Region', 'Car': 'Parking', 'Propertycount': 'SuburbPropCount'})
df.head()

## Duplicates

In [ ]:
# check for duplicated entries
s = df.duplicated(keep = False)
s = s[s == True]
s

In [ ]:
df.loc[[7769,7770]]

In [ ]:
# lets drop one of them in the dataframe
df.drop([7769], inplace=True)

## Abbreviated Categories

In [ ]:
# Let's have a look at the features with missing categories compared to the definition
df['Method'].unique()

We have:<br>
        S - property sold; <br>
        SP - property sold prior; <br>
        PI - property passed in; <br>
        VB - vendor bid; <br>
        SA - sold after auction; 

We are missing:<br>
        W - withdrawn prior to auction; <br>
        SS - sold after auction price not disclosed. <br>
        N/A - price or highest bid not available.<br>
        PN - sold prior not disclosed; <br>
        SN - sold not disclosed; <br>
        NB - no bid; 

Apparently, there has been already some cleaning being done on this dataset and the entries of unsold houses or houses without a price have been discarded.
       

In [ ]:
# Let's have a look at the features with missing categories compared to the definition
df['Type'].unique()

We have:<br>
           h - house,cottage,villa,semi,terrace; <br>
           u - unit, duplex; <br>
           t - townhouse; 
           
We do not have:       
           br - bedroom(s); <br>
           dev site - development site; <br>
           o res - other residential.

Apparently also here, the data was precleaned, removing single bedroom offers, developement sites and others, leaving us with houses, units and townhouses.

In [ ]:
# Let's replace abbreviations
df = df.replace({'Type': {'h':'house','u':'unit','t':'townhouse'}})
df = df.replace({'Method': {'S':'Property Sold','SP':'Property Sold Prior','PI':'Property Passed In',
                            'VB':'Vendor Bid', 'SA':'Sold After Auction'}})
df.head()

## Postcodes vs Suburbs

In [ ]:
#One might think, what about the postcode and the suburb, how are they connected. Let's have a look:
postcodes_df = df.groupby('Postcode', as_index=False).Suburb.nunique()
postcodes_df.columns = ['Postcode', '#Assigned Suburbs']
postcodes_df.loc[postcodes_df['#Assigned Suburbs'] > 1]

In [ ]:
postcodes_df.loc[postcodes_df['#Assigned Suburbs'] > 1].count()

Of 198 postcodes, 73 are used for multiple suburbs. still, the postcodes are a subset of the suburbs and probably not necessary. Judging by this, let us remove the postcode for now.

In [ ]:
df = df.drop(['Postcode'],axis=1)
df.head()

## Categorical Features

In [ ]:
df['CouncilArea'].unique()

We see there is a category called "Unavailable" and the missing values labelled with None. We will come back to this later.

In [ ]:
df['Suburb'].unique()

Normally, at this point we should use fuzzy matching techniques to see, if there are duplicate entries that are written similarly or have a typo or a space somewhere, but we leave it at this for now.

# Statistical Analysis

In [ ]:
# show statistical properties for the numerical features, the lambda converts values into a more readable format (float is shown in scientific notion)
dist_df = df.describe().T.apply(lambda s: s.apply(lambda x: format(x, 'g')))
dist_df

In [ ]:
# Let's add some other information missing to the statistics
from pandas.api.types import is_numeric_dtype
max_count=[]
min_count=[]
mode_count=[]
mode=[]
skew=[]
for cl in df.columns:
    if (is_numeric_dtype(df[cl])):
        max_count.append(df[cl].value_counts(dropna=False).loc[df[cl].max()])
        min_count.append(df[cl].value_counts(dropna=False).loc[df[cl].min()])
        mode_count.append(df[cl].value_counts(dropna=False).loc[df[cl].mode()[0]])
        skew.append(df[cl].skew())
        mode.append(int(df[cl].mode()[0]))

dist_df['mode'] = mode
dist_df['skew'] = skew
dist_df['#values(min)'] = min_count
dist_df['#values(max)'] = max_count
dist_df['#values(mode)'] = mode_count
dist_df


Points of interest for further analysis:

- Price: Skewed to the right, we will probably see some few high prices. Not surprising.
- Distance: Skewed to the right, probably due to the one sample being 48.1km away from the CBD in Melbourne. Interestingly enough there are 6 samples with 0 distance. Sometimes 0 is a dummy value, so we should check that out. Judging by the fact that the mode 11 is set 739 times, the distance might not be exactly the distance from the city center, but the mean distance of a suburb perhaps from the city center. We should check this out.
- Bedrooms: Skewed to the right, due to some high amounts of bedrooms in some places. Curiously there are 16 samples with 0 bedrooms, which needs to be checked.
- Bathrooms: same as for bedrooms, with 34 samples of 0 bathrooms, which sounds weird.
- Parking: same as for bedrooms, but here 1026 samples with no parking spaces is not surprising.
- Landsize: Extremely skewed (95.24) to the right. The max is 433014. If we presume m2, these are about 43 hectar of land. Not impossible, but it probably would distort our modelling. 
- BuildingArea: Also extremely skewed to the right, due to the maximum size of 44515 m2. This sounds improbable, so we might want to remove that one. Also there are 17 samples with 0 building area, which also does not sound good.
- YearBuilt: skewed to the left due to the one building built in 1196.
- SuburbPropCount: slightly skewed the right. We have to see how helpful this value is.

Let's go through these points:

## Price

In [ ]:
fig = px.box(df, x="Price",points="all")
fig.show()

As we presumed, still a bunch of prices past the upper fence (2.35M). Lets create a new field as the log of Price.

In [ ]:
df["Price_log"] = np.log(df['Price'])

In [ ]:
fig = px.box(df, x="Price_log",points="all")
fig.show()

That looks far more natural. Let's follow this one throughout.

## Distance

In [ ]:
df.loc[df['Distance'] == 0]

## Bedrooms

In [ ]:
df.loc[df['Bedrooms'] == 0]

## BuildingArea

In [ ]:
fig = px.box(df, y="BuildingArea",points="all")
fig.show()

A very distorted box plot. Hovering over the box you can see the statistics. The upper fence is at 295, which means statistically (presuming a normal distribution) everything above that is an outlier. Lets have a look at them.

In [ ]:
df.loc[raw_df['BuildingArea'] > 295]['BuildingArea'].count()

Still a lot of samples in this.

In [ ]:
# checking area over 2000 leaves us with 4 examples
df.loc[raw_df['BuildingArea'] > 2000]

As we can see the last house is 48.1 km or miles away from the city center, therefore having a landsize and building area in those values are feasible. Though if we want to understand house prices in Melbourne, this might not be that important. It is also in the Northern Victoria region and not in the metropolitan regions. We could go further here to have a look in the connection between these specific houses outside of the norm in conjunction with other features, but we will leave it at this for now.

In [ ]:
df.drop([13245], inplace=True)

In [ ]:
df.loc[raw_df['BuildingArea'] > 2000]

## Landsize

In [ ]:
# checking Landsize
fig = px.box(df, y="Landsize",points="all")
fig.show()

The same seems to be true for Landsize. The upper fence is at 1357, but we even see one outlier over 400000.

In [ ]:
df.loc[raw_df['Landsize'] > 1357]['Landsize'].count()

Still a lot of houses in this one.

In [ ]:
# let check for a higher number to get some samples
df.loc[raw_df['Landsize'] > 30000]

Before we move on, let us store our first cleansing as a dataset in Azure ML

In [ ]:
Dataset.Tabular.register_pandas_dataframe(dataframe = df, target = datastore, name ='Melbourne Housing Dataset', description = 'Data Cleansing 1 - removed address, postcode, duplicates and outliers')

# Missing Values and Correlations

In [ ]:
# If we would drop any row with an empty value, we would loose half the dataset (6196 rows left). Lets look at the row distribution of missing values.
df.dropna(how='any').shape

In [ ]:
import missingno as msno
msno.matrix(df);

The missing values for CouncilArea seem to be at the end of the list, the one for parking also very localized, the others ones are all over the place.

## CouncilArea

In [ ]:
df['CouncilArea'].unique()

In [ ]:
df.loc[df.CouncilArea.isin(['Unavailable'])]

In [ ]:
df['CouncilArea'].fillna(value = "Missing", inplace = True)
df['CouncilArea'].replace(to_replace="Unavailable", value="Missing", inplace=True)

In [ ]:
df['CouncilArea'].unique()

A better way would be to find a list of CouncilAreas and fill the correct values in it.

## BuildingArea

In [ ]:
# simplest way, replace the building area by the mean value of the other entries
BA_mean = df['BuildingArea'].mean()
df['BuildingArea'].replace(to_replace=np.nan, value=BA_mean, inplace=True)
df['BuildingArea'].isnull().sum()
BA_mean

## YearBuilt

In [ ]:
YB_median = df['YearBuilt'].median()
df['YearBuilt'].replace(to_replace=np.nan, value=YB_median, inplace=True)
df['YearBuilt'].isnull().sum()
YB_median

## Parking

In [ ]:
PK_median = df['Parking'].median()
df['Parking'].replace(to_replace=np.nan, value=PK_median, inplace=True)
df['Parking'].isnull().sum()
PK_median

In [ ]:
Dataset.Tabular.register_pandas_dataframe(dataframe = df, target = datastore, name ='Melbourne Housing Dataset', description = 'Data Cleansing 2 - replaced missing values')

# Correlations

Let us have a look at the correlation between different features with a correlation matrix.

In [ ]:
# compute the correlation matrix
corr = df.corr()

# generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))
# set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

As we can see, we are missing features. There are 13 features shown, which are all the numerical columns. This means, the algorithm cannot handle our objects and datetime columns. Let us change that

## Converting Categorical Types

In [ ]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

In [ ]:
# Let's convert all columns in the object dataframe to the "categorical" datatype
for cl in obj_df.columns:
    obj_df[cl] = obj_df[cl].astype('category')
    
obj_df.dtypes

In [ ]:
for cl in obj_df.columns:
    obj_df[cl+"_cat"] = obj_df[cl].cat.codes
obj_df.head()

In [ ]:
# Now we will incorporate the numbered versions of our categorical data into a new dataframe
column_replacement = {'Type':'Type_cat','Suburb':'Suburb_cat','Method':'Method_cat','CouncilArea':'CouncilArea_cat','Region':'Region_cat'}
cont_df = df.copy()
for key in column_replacement:
    cont_df[key] = obj_df[column_replacement[key]]
cont_df.dtypes

In [ ]:
cont_df['Date_Epoch'] = cont_df['Date'].apply(lambda x: x.timestamp())
cont_df.drop(['Date'], axis=1, inplace=True)
cont_df.dtypes

In [ ]:
for cl in cont_df.columns:
    if (cont_df[cl].dtype == np.float64 and cl not in ['Lattitude', 'Longtitude', 'Price_log', 'Distance']):
        cont_df[cl] = cont_df[cl].astype('int')
cont_df.dtypes

## Correlation with converted categories

In [ ]:
# Let us do the correlation again
# Compute the correlation matrix
corr = cont_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

What can we see:

- Rooms is strongly correlated with Price, Price_log, Distance, Bedrooms, Bathrooms, Parking and Building Area
- Type is strongly correlated with Price, Price_log, Bedrooms, YearBuilt and Rooms
- Price is strongly correlated with Rooms, Type, Bedrooms, Bathrooms, Parking BuildingArea
- Suburb, Method, Landsize and SuburbPropCount seem not to have to much influence in its current state on other features or the target


In [ ]:
Dataset.Tabular.register_pandas_dataframe(dataframe = cont_df, target = datastore, name ='Melbourne Housing Dataset', description = 'Data Cleansing 3 - all features converted to numerical values')

# Feature Importance

## Price as the Target

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

# create X and Y vectors
ignored_col = ['Price', 'Price_log', 'Date']
cols = [c for c in cont_df.columns if c not in ignored_col]
X = cont_df[cols]
y = cont_df['Price']

# define the regression forest
forest = ExtraTreesRegressor(n_estimators=250, criterion='mse', random_state=0, max_depth=10)

# run the forest fitting with our vectors
forest.fit(X, y)

# create indices order for feature importance
importances = forest.feature_importances_
tree_importances = np.array([tree.feature_importances_ for tree in forest.estimators_])
indices = np.argsort(importances)[::-1]

imp_df = pd.DataFrame([tree.feature_importances_ for tree in forest.estimators_], columns = cols)

# sort columns by importance
imp_df = imp_df[[cols[i] for i in indices.tolist()]]
imp_df.head()

In [ ]:
# creating barplot to visualize feature importance
sns.set(style="whitegrid")
palette = sns.color_palette(n_colors=3)
fig = plt.figure()

ax = sns.barplot(data=imp_df, color=palette[1], capsize=.2, errwidth=1.2)
plt.title("Feature importances")
plt.xticks(range(X.shape[1]), rotation='vertical')
plt.show()

## log(Price) Target

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

# create X and Y vectors
ignored_col = ['Price', 'Price_log', 'Date']
cols = [c for c in cont_df.columns if c not in ignored_col]
X = cont_df[cols]
y = cont_df['Price_log']

# define the regression forest
forest = ExtraTreesRegressor(n_estimators=250, criterion='mse', random_state=0, max_depth=10)

# run the forest fitting with our vectors
forest.fit(X, y)

# create indices order for feature importance
importances = forest.feature_importances_
tree_importances = np.array([tree.feature_importances_ for tree in forest.estimators_])
indices = np.argsort(importances)[::-1]

imp_df = pd.DataFrame([tree.feature_importances_ for tree in forest.estimators_], columns = cols)

# sort columns by importance
imp_df = imp_df[[cols[i] for i in indices.tolist()]]
imp_df.head()

In [ ]:
sns.set(style="whitegrid")
palette = sns.color_palette(n_colors=3)
fig = plt.figure()

ax = sns.barplot(data=imp_df, color=palette[1], capsize=.2, errwidth=1.2)
plt.title("Feature importances")
plt.xticks(range(X.shape[1]), rotation='vertical')
plt.show()

## Digging deeper on correlation between Price and Type

In [ ]:
fig = px.box(df, y="Price_log",x='Type', color = 'Type', 
                 category_orders={"Type": ["house", "townhouse", "unit"]})
fig.show()